
---
<p style="text-align: center;">
<font size="5"> <b>Data Wrangling For Tableau Dashboard<b> </font> </p>

---

##### Libraries

In [1]:
import pandas as pd #To data wrangling
import re #To create Regular expressions

Lets **import the dataframe** that we got after the simluation to plot the previous efficient frontier

In [5]:
initial_dataframe = pd.read_csv(r'C:\Users\calni\OneDrive\Documents\Coding\Projects data\Ef Dash\EficcientFrontierRecession.csv')
print(initial_dataframe.head())

     Return      Risk    Sharpe                          Weights
0  0.175990  0.157749  1.115630  [0.184 0.445 0.108 0.049 0.214]
1  0.171697  0.151920  1.130180  [0.105 0.285 0.1   0.255 0.255]
2  0.180241  0.157031  1.147807  [0.13  0.077 0.147 0.275 0.371]
3  0.149156  0.144305  1.033616  [0.298 0.246 0.249 0.166 0.042]
4  0.165245  0.149631  1.104350  [0.106 0.312 0.298 0.088 0.197]


In [15]:
#Checking the columns data type
print(type(initial_dataframe.Weights[0]))
print(type(initial_dataframe.Risk[0]))

<class 'str'>
<class 'numpy.float64'>


As we can see the column **'Weights'** doesnt contain numerical data type, instead its data is a **string**.
Additionally, in order to plot the weights composition in Tableau, this dataframe is not in the most
suitable way to do it, thats why **we need to arrange the data** before uploading it to Tableau.

**Lets see how**.

First, **lets get the top 10 portfolios with the highest sharpe ratio**
to subset the dataframe.

In [17]:
topten = initial_dataframe.nlargest(n=10, columns=['Sharpe'])
#To reset the index of the portfolios
topten.reset_index(inplace=True)
#To delete the old index column
del topten['index']
print(topten)

     Return      Risk    Sharpe                          Weights
0  0.207376  0.174751  1.186695  [0.187 0.187 0.071 0.037 0.518]
1  0.211138  0.177933  1.186616  [0.207 0.158 0.053 0.034 0.548]
2  0.210402  0.177324  1.186541  [0.207 0.19  0.062 0.008 0.533]
3  0.202084  0.170459  1.185531  [0.26  0.151 0.083 0.038 0.468]
4  0.204565  0.172577  1.185357  [0.272 0.19  0.047 0.02  0.471]
5  0.213353  0.179993  1.185338  [0.213 0.19  0.004 0.038 0.554]
6  0.209331  0.176682  1.184790  [0.294 0.17  0.026 0.005 0.505]
7  0.206081  0.173960  1.184642  [0.227 0.13  0.016 0.113 0.514]
8  0.214825  0.181377  1.184413  [0.169 0.166 0.046 0.035 0.584]
9  0.197300  0.166587  1.184372  [0.191 0.178 0.106 0.077 0.447]


Now we need to create a dataframe with:
- Name of the portfolio - i.e (Portfolio 1,2,3,4)
- Sharpe ratio
- Corresponding weights for each stock - i.e (WM,MCD,PG,JNJ,HD)

Lest see how

**To begin** we need to **make the list of weights as float not as string**

In [21]:
#Regular expression to match the string pattern
#in the weights column and put it in a empty list
xe = re.compile(r'\d\.\d+')

#Empty list
weights = []

#Matching the string and appending the weights 
#to the new empty list

#To iterate over 10 times, since there are 10 portfolios
for i in range (len(topten['Weights'])):
   #To match the string pattern and put it in a list
   matches = xe.findall(topten['Weights'][i])
   #To change the matches from string to float data type
   for j,k in enumerate(matches):
      matches[j] = float(k)
   #To append the list to the new weights list
   weights.append(matches)

print(weights)
print(type(weights[0][0]))

[[0.187, 0.187, 0.071, 0.037, 0.518], [0.207, 0.158, 0.053, 0.034, 0.548], [0.207, 0.19, 0.062, 0.008, 0.533], [0.26, 0.151, 0.083, 0.038, 0.468], [0.272, 0.19, 0.047, 0.02, 0.471], [0.213, 0.19, 0.004, 0.038, 0.554], [0.294, 0.17, 0.026, 0.005, 0.505], [0.227, 0.13, 0.016, 0.113, 0.514], [0.169, 0.166, 0.046, 0.035, 0.584], [0.191, 0.178, 0.106, 0.077, 0.447]]
<class 'float'>


**Getting the Sharpes Column**

In [25]:
sharpes = []
for i in topten['Sharpe']:
   sharpes.append(i)

print(sharpes)
print(type(sharpes[0]))

[1.1866945768751307, 1.1866164100416663, 1.1865409389426642, 1.185530785574375, 1.185356946094067, 1.1853378279261773, 1.184789821147318, 1.1846423914691255, 1.184412560479264, 1.1843718496311624]
<class 'float'>


**Creating the other columns for the new dataframe**

In [30]:
wmr = []
mcdr = []
pgr = []
jnj=[]
hd=[]
name = ['Pt 1','Pt 2','Pt 3','Pt 4','Pt 5','Pt 6','Pt 7','Pt 8','Pt 9','Pt 10']

**Appending the corresponding weight to its column**

In [31]:
for i in weights:
   wmr.append(i[0])
   mcdr.append(i[1])
   pgr.append(i[2])
   jnj.append(i[3])
   hd.append(i[4])

print(wmr)

[0.187, 0.207, 0.207, 0.26, 0.272, 0.213, 0.294, 0.227, 0.169, 0.191]


**Putting all together**

In [32]:
df = [name,sharpes,wmr,mcdr,pgr,jnj,hd]
finaldf = pd.DataFrame(df).T
finaldf.columns = ['Names','Portfolio Sharpe','WM','MCD','PG','JNJ','HD']

print(finaldf)

   Names Portfolio Sharpe     WM    MCD     PG    JNJ     HD
0   Pt 1         1.186695  0.187  0.187  0.071  0.037  0.518
1   Pt 2         1.186616  0.207  0.158  0.053  0.034  0.548
2   Pt 3         1.186541  0.207   0.19  0.062  0.008  0.533
3   Pt 4         1.185531   0.26  0.151  0.083  0.038  0.468
4   Pt 5         1.185357  0.272   0.19  0.047   0.02  0.471
5   Pt 6         1.185338  0.213   0.19  0.004  0.038  0.554
6   Pt 7          1.18479  0.294   0.17  0.026  0.005  0.505
7   Pt 8         1.184642  0.227   0.13  0.016  0.113  0.514
8   Pt 9         1.184413  0.169  0.166  0.046  0.035  0.584
9  Pt 10         1.184372  0.191  0.178  0.106  0.077  0.447


**And thats the final dataframe that was uploaded to tableau, in order to make this dashboard**

[Link to Tableau Interactive Dashboard](https://public.tableau.com/app/profile/camilo.alonso/viz/EfRecess/Dashboard1)


<img src='EF_Frontier.png' width='1074' height='750'/>